# Split

In [1]:
import pandas as pd
import os
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.linear_model import LinearRegression

os.sys.path.append('./..')

from hyperopt.pyll import scope
from utils.loadgenius import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import precision_score
import mlflow
from utils.pretty import pprint
from functools import wraps
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


In [2]:
numericals = ["n_months", "n_origins", "total_teu",
             "avg_teu", "n_goods", "n_ports", "n_states",
             "biggest_gap", "smallest_gap", "n_zipcodes", "n_istates"]

categoricals = ["major_origin", "major_goods", "major_port_state"]

target = 'importer_rank_cate'

df = DataLoader("../catalog.yml").load('ml_dataset')
pprint(f"Retrieved dataset: {df.shape}")


Retrieved dataset: (20674, 16)

In [3]:
# Split data into train and validation sets (80-20, stratified)
def split_data(X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, 
        y,
        test_size=0.2,
        stratify=y,
        random_state=42
    )
    return X_train, X_val, y_train, y_val


In [4]:
# # OneHot encoding
# X = pd.get_dummies(
#     df.drop(target, axis=1),
#     columns=categoricals
# )
# y = df[target]


# X_train, X_val, y_train, y_val = split_data(X, y)
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dval = xgb.DMatrix(X_val, label=y_val)

In [6]:
# Objective function decorator
def objective(func):
    @wraps(func)
    def wrapper(params):
        
        with mlflow.start_run():
            y_val_pred = func(params)
            # Calculate precision
            precision = precision_score(y_val, y_val_pred, average='weighted')

        return {'loss': 1 - precision, 'status': STATUS_OK}



    return wrapper

# Define the objective function for Gradient Boosting

@objective
def gradient_boosting_objective(params):
    
    mlflow.set_tag("model", "gb")
    mlflow.log_params(params)

    # Initialize and train the Gradient Boosting classifier
    clf = GradientBoostingClassifier(**params, random_state=0)
    clf.fit(X_train, y_train)
    
    return clf.predict(X_val)        


# Define the objective function for XGBoost


@objective
def xgb_objective(params):
    
    mlflow.set_tag("model", "xgb")
    mlflow.log_params(params)
    
    booster = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=100,
        evals=[(dval, 'validation')],
        early_stopping_rounds=50
    )

    return booster.predict(dval)        

@objective
def lr_objective(params):
    mlflow.set_tag("model", "lr")
    mlflow.log_params(params)

    # Initialize and train the Softmax (Logistic Regression) classifier
    lr = LogisticRegression(
        multi_class='multinomial',
        **params
    )
    lr.fit(X_train, y_train)

    return lr.predict(X_val)

@objective
def dt_objective(params):
    mlflow.set_tag("model", "dt")
    mlflow.log_params(params)

    # Initialize and train the Decision Tree classifier
    clf = DecisionTreeClassifier(**params, random_state=0)
    clf.fit(X_train, y_train)

    return clf.predict(X_val)


In [25]:
# Define hyperparameter space for hyperopt
xgb_space = {
    'objective': 'multi:softmax',
    'learning_rate': hp.loguniform('learning_rate', -3, -1),
    'max_depth': scope.int(hp.quniform('max_depth', 2, 5, 1)),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'subsample': hp.uniform('subsample', 0.5, .9),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'num_class': 4
    }

gb_space = {
    'learning_rate': hp.loguniform('learning_rate', -4, -1),
    'max_depth': scope.int(hp.quniform('max_depth', 1, 2, 1)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'n_estimators': scope.int(hp.quniform('n_estimators', 10, 30, 1)),
    }

# Define hyperparameter space for hyperopt
lr_space = {
    'n_jobs': -1,
    'penalty': "l2",
    'max_iter': int(1e8)
}


# Define the search space for Decision Tree
dt_space = {
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    'max_depth': scope.int(hp.uniform('max_depth', 7, 15)),
    'min_samples_split': hp.loguniform('min_samples_split', -4, -1),
    'class_weight': 'balanced'
    }

In [8]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("importer-class-experiment")



2023/11/22 23:19:48 INFO mlflow.tracking.fluent: Experiment with name 'importer-class-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/tunglinux/units/python_practice/impgenius/notebooks/mlruns/1', creation_time=1700669988140, experiment_id='1', last_update_time=1700669988140, lifecycle_stage='active', name='importer-class-experiment', tags={}>

# With numeric features only

In [18]:
X = df[numericals]
y = df[target]


X_train, X_val, y_train, y_val = split_data(X, y)
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

y_train = y_train.ravel()
y_val = y_val.ravel()

In [10]:
# Model 1: xgb
xgb_best_result = fmin(
    fn=xgb_objective,
    space=xgb_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)


[0]	validation-mlogloss:1.36032                                                  
[1]	validation-mlogloss:1.34125                                                  
[2]	validation-mlogloss:1.32857                                                  
[3]	validation-mlogloss:1.31787                                                  
[4]	validation-mlogloss:1.30969                                                  
[5]	validation-mlogloss:1.30357                                                  
[6]	validation-mlogloss:1.29932                                                  
[7]	validation-mlogloss:1.29559                                                  
[8]	validation-mlogloss:1.29312                                                  
[9]	validation-mlogloss:1.29098                                                  
[10]	validation-mlogloss:1.28915                                                 
[11]	validation-mlogloss:1.28767                                                 
[12]	validation-

[22]	validation-mlogloss:1.28476                                                 
[23]	validation-mlogloss:1.28477                                                 
[24]	validation-mlogloss:1.28482                                                 
[25]	validation-mlogloss:1.28470                                                 
[26]	validation-mlogloss:1.28468                                                 
[27]	validation-mlogloss:1.28475                                                 
[28]	validation-mlogloss:1.28479                                                 
[29]	validation-mlogloss:1.28425                                                 
[30]	validation-mlogloss:1.28442                                                 
[31]	validation-mlogloss:1.28446                                                 
[32]	validation-mlogloss:1.28433                                                 
[33]	validation-mlogloss:1.28411                                                 
[34]	validation-

[27]	validation-mlogloss:1.28837                                                 
[28]	validation-mlogloss:1.28818                                                 
[29]	validation-mlogloss:1.28801                                                 
[30]	validation-mlogloss:1.28781                                                 
[31]	validation-mlogloss:1.28758                                                 
[32]	validation-mlogloss:1.28730                                                 
[33]	validation-mlogloss:1.28715                                                 
[34]	validation-mlogloss:1.28704                                                 
[35]	validation-mlogloss:1.28688                                                 
[36]	validation-mlogloss:1.28683                                                 
[37]	validation-mlogloss:1.28655                                                 
[38]	validation-mlogloss:1.28640                                                 
[39]	validation-

[26]	validation-mlogloss:1.28366                                                 
[27]	validation-mlogloss:1.28393                                                 
[28]	validation-mlogloss:1.28412                                                 
[29]	validation-mlogloss:1.28412                                                 
[30]	validation-mlogloss:1.28409                                                 
[31]	validation-mlogloss:1.28373                                                 
[32]	validation-mlogloss:1.28389                                                 
[33]	validation-mlogloss:1.28361                                                 
[34]	validation-mlogloss:1.28375                                                 
[35]	validation-mlogloss:1.28394                                                 
[36]	validation-mlogloss:1.28383                                                 
[37]	validation-mlogloss:1.28380                                                 
[38]	validation-

[34]	validation-mlogloss:1.28349                                                 
[35]	validation-mlogloss:1.28359                                                 
[36]	validation-mlogloss:1.28331                                                 
[37]	validation-mlogloss:1.28321                                                 
[38]	validation-mlogloss:1.28335                                                 
[39]	validation-mlogloss:1.28312                                                 
[40]	validation-mlogloss:1.28284                                                 
[41]	validation-mlogloss:1.28280                                                 
[42]	validation-mlogloss:1.28266                                                 
[43]	validation-mlogloss:1.28266                                                 
[44]	validation-mlogloss:1.28245                                                 
[45]	validation-mlogloss:1.28247                                                 
[46]	validation-

...

[51]	validation-mlogloss:1.28924                                                 
[52]	validation-mlogloss:1.28887                                                 
[53]	validation-mlogloss:1.28868                                                 
[54]	validation-mlogloss:1.28837                                                 
[55]	validation-mlogloss:1.28812                                                 
[56]	validation-mlogloss:1.28801                                                 
[57]	validation-mlogloss:1.28771                                                 
[58]	validation-mlogloss:1.28744                                                 
[59]	validation-mlogloss:1.28708                                                 
[60]	validation-mlogloss:1.28681                                                 
[61]	validation-mlogloss:1.28656                                                 
[62]	validation-mlogloss:1.28636                                                 
[63]	validation-

[50]	validation-mlogloss:1.28514                                                 
[51]	validation-mlogloss:1.28513                                                 
[52]	validation-mlogloss:1.28502                                                 
[53]	validation-mlogloss:1.28490                                                 
[54]	validation-mlogloss:1.28499                                                 
[55]	validation-mlogloss:1.28486                                                 
[56]	validation-mlogloss:1.28507                                                 
[57]	validation-mlogloss:1.28517                                                 
[58]	validation-mlogloss:1.28511                                                 
[59]	validation-mlogloss:1.28512                                                 
[60]	validation-mlogloss:1.28513                                                 
[61]	validation-mlogloss:1.28511                                                 
[62]	validation-

[49]	validation-mlogloss:1.28717                                                 
[50]	validation-mlogloss:1.28705                                                 
[51]	validation-mlogloss:1.28750                                                 
[52]	validation-mlogloss:1.28787                                                 
[53]	validation-mlogloss:1.28820                                                 
[54]	validation-mlogloss:1.28835                                                 
[55]	validation-mlogloss:1.28846                                                 
[56]	validation-mlogloss:1.28833                                                 
[57]	validation-mlogloss:1.28806                                                 
[58]	validation-mlogloss:1.28788                                                 
[59]	validation-mlogloss:1.28807                                                 
[60]	validation-mlogloss:1.28812                                                 
[61]	validation-

[81]	validation-mlogloss:1.28607                                                 
[82]	validation-mlogloss:1.28595                                                 
[83]	validation-mlogloss:1.28601                                                 
[84]	validation-mlogloss:1.28624                                                 
[85]	validation-mlogloss:1.28604                                                 
[86]	validation-mlogloss:1.28590                                                 
[87]	validation-mlogloss:1.28615                                                 
[88]	validation-mlogloss:1.28614                                                 
[89]	validation-mlogloss:1.28610                                                 
[90]	validation-mlogloss:1.28622                                                 
[91]	validation-mlogloss:1.28631                                                 
[92]	validation-mlogloss:1.28644                                                 
[0]	validation-m

[87]	validation-mlogloss:1.28518                                                 
[88]	validation-mlogloss:1.28521                                                 
[89]	validation-mlogloss:1.28531                                                 
[90]	validation-mlogloss:1.28526                                                 
[91]	validation-mlogloss:1.28524                                                 
[92]	validation-mlogloss:1.28512                                                 
[93]	validation-mlogloss:1.28507                                                 
[94]	validation-mlogloss:1.28504                                                 
[95]	validation-mlogloss:1.28499                                                 
[96]	validation-mlogloss:1.28486                                                 
[97]	validation-mlogloss:1.28476                                                 
[98]	validation-mlogloss:1.28471                                                 
[99]	validation-

In [11]:
import hyperopt
# Train the final model with the best hyperparameters
xgb_best = xgb.train(
    params=hyperopt.space_eval(xgb_space, xgb_best_result),
    dtrain=dtrain, 
    num_boost_round=100,
    evals=[(dval, 'validation')])

# Predict on the validation set with the best model
y_val_pred_xgb = xgb_best.predict(dval).astype(int)

# Generate the classification report
xgb_report = classification_report(y_val, y_val_pred_xgb)
print(xgb_report)


[0]	validation-mlogloss:1.37878
[1]	validation-mlogloss:1.37199
[2]	validation-mlogloss:1.36593
[3]	validation-mlogloss:1.36013
[4]	validation-mlogloss:1.35479
[5]	validation-mlogloss:1.34971
[6]	validation-mlogloss:1.34546
[7]	validation-mlogloss:1.34129
[8]	validation-mlogloss:1.33758
[9]	validation-mlogloss:1.33396
[10]	validation-mlogloss:1.33081
[11]	validation-mlogloss:1.32799
[12]	validation-mlogloss:1.32523
[13]	validation-mlogloss:1.32272
[14]	validation-mlogloss:1.32075
[15]	validation-mlogloss:1.31863
[16]	validation-mlogloss:1.31660
[17]	validation-mlogloss:1.31474
[18]	validation-mlogloss:1.31311
[19]	validation-mlogloss:1.31150
[20]	validation-mlogloss:1.31006
[21]	validation-mlogloss:1.30869
[22]	validation-mlogloss:1.30747
[23]	validation-mlogloss:1.30633
[24]	validation-mlogloss:1.30520
[25]	validation-mlogloss:1.30419
[26]	validation-mlogloss:1.30316
[27]	validation-mlogloss:1.30223
[28]	validation-mlogloss:1.30139
[29]	validation-mlogloss:1.30059
[30]	validation-mlog

In [12]:
# Model 2: gb
gb_best_result = fmin(
    fn=gradient_boosting_objective,
    space=gb_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)


 22%|██▏       | 11/50 [00:09<00:30,  1.27trial/s, best loss: 0.5879142941954514]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 34%|███▍      | 17/50 [00:15<00:30,  1.10trial/s, best loss: 0.5879142941954514]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 42%|████▏     | 21/50 [00:19<00:27,  1.06trial/s, best loss: 0.5870592975319289]

...

 82%|████████▏ | 41/50 [00:36<00:06,  1.29trial/s, best loss: 0.5870592975319289]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 92%|█████████▏| 46/50 [00:41<00:03,  1.22trial/s, best loss: 0.5870592975319289]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



100%|██████████| 50/50 [00:45<00:00,  1.10trial/s, best loss: 0.5870592975319289]


In [13]:
# Retrieve the best hyperparameters from the Trials object
gb_best_params = hyperopt.space_eval(gb_space, gb_best_result)

# Train the final model with the best hyperparameters
gb_best = GradientBoostingClassifier(random_state=0, **gb_best_params)
gb_best.fit(X_train, y_train)

# Predict on the validation set with the best model
y_val_pred_gb = gb_best.predict(X_val).astype(int)

# Generate the classification report
gb_report = classification_report(y_val, y_val_pred_gb)
print(gb_report)



              precision    recall  f1-score   support

           0       0.68      0.08      0.14       559
           1       0.41      0.01      0.02       880
           2       0.33      0.05      0.08      1185
           3       0.38      0.98      0.55      1511

    accuracy                           0.39      4135
   macro avg       0.45      0.28      0.20      4135
weighted avg       0.41      0.39      0.25      4135



In [14]:
# Model 3: lr
lr_best_result = fmin(
    fn=lr_objective,
    space=lr_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)


100%|██████████| 50/50 [04:53<00:00,  5.87s/trial, best loss: 0.6177271752360818]


In [15]:
# Retrieve the best hyperparameters from the Trials object
lr_best_params = hyperopt.space_eval(lr_space, lr_best_result)

# Train the final model with the best hyperparameters
lr_best = LogisticRegression(random_state=0, **lr_best_params)
lr_best.fit(X_train, y_train)

# Predict on the validation set with the best model
y_val_pred_lr = lr_best.predict(X_val).astype(int)

# Generate the classification report
lr_report = classification_report(y_val, y_val_pred_lr)
print(lr_report)



              precision    recall  f1-score   support

           0       0.60      0.10      0.17       559
           1       0.30      0.08      0.12       880
           2       0.32      0.08      0.13      1185
           3       0.40      0.93      0.56      1511

    accuracy                           0.39      4135
   macro avg       0.40      0.30      0.24      4135
weighted avg       0.38      0.39      0.29      4135



In [26]:
# Model 4: dt
dt_best_result = fmin(
    fn=dt_objective,
    space=dt_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)


 12%|█▎         | 6/50 [00:00<00:04, 10.42trial/s, best loss: 0.6597434073048168]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 20%|██        | 10/50 [00:00<00:03, 10.39trial/s, best loss: 0.6594417211074189]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 28%|██▊       | 14/50 [00:01<00:03, 10.07trial/s, best loss: 0.6594417211074189]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 88%|████████▊ | 44/50 [00:04<00:00,  9.27trial/s, best loss: 0.6563790232148397]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



100%|██████████| 50/50 [00:05<00:00,  9.63trial/s, best loss: 0.6558752135704268]


In [27]:
dt_best_params = hyperopt.space_eval(dt_space, dt_best_result)

# Train the final model with the best hyperparameters
dt_best = DecisionTreeClassifier(random_state=0, **dt_best_params)
dt_best.fit(X_train, y_train)

# Predict on the validation set with the best model
y_val_pred_dt = dt_best.predict(X_val).astype(int)

# Generate the classification report
dt_report = classification_report(y_val, y_val_pred_dt)
print(dt_report)



              precision    recall  f1-score   support

           0       0.24      0.41      0.31       559
           1       0.27      0.10      0.14       880
           2       0.35      0.02      0.04      1185
           3       0.42      0.77      0.54      1511

    accuracy                           0.37      4135
   macro avg       0.32      0.33      0.26      4135
weighted avg       0.34      0.37      0.28      4135



# With categorical variables

In [36]:
# OneHot encoding
X = pd.get_dummies(
    df.drop([target, 'importer_rank'], axis=1),
    columns=categoricals
)
y = df[target]

X_train, X_val, y_train, y_val = split_data(X, y)
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

y_train = y_train.ravel()
y_val = y_val.ravel()

In [39]:
# Model 1: xgb
xgb_best_result = fmin(
    fn=xgb_objective,
    space=xgb_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)


[0]	validation-mlogloss:1.37290                                                  
[1]	validation-mlogloss:1.36078                                                  
[2]	validation-mlogloss:1.35023                                                  
[3]	validation-mlogloss:1.34072                                                  
[4]	validation-mlogloss:1.33296                                                  
[5]	validation-mlogloss:1.32589                                                  
[6]	validation-mlogloss:1.32000                                                  
[7]	validation-mlogloss:1.31427                                                  
[8]	validation-mlogloss:1.30951                                                  
[9]	validation-mlogloss:1.30505                                                  
[10]	validation-mlogloss:1.30153                                                 
[11]	validation-mlogloss:1.29818                                                 
[12]	validation-

[99]	validation-mlogloss:1.25106                                                 
[0]	validation-mlogloss:1.35883                                                  
[1]	validation-mlogloss:1.33990                                                  
[2]	validation-mlogloss:1.32490                                                  
[3]	validation-mlogloss:1.31457                                                  
[4]	validation-mlogloss:1.30633                                                  
[5]	validation-mlogloss:1.29973                                                  
[6]	validation-mlogloss:1.29453                                                  
[7]	validation-mlogloss:1.28973                                                  
[8]	validation-mlogloss:1.28640                                                  
[9]	validation-mlogloss:1.28278                                                  
[10]	validation-mlogloss:1.28045                                                 
[11]	validation-

[98]	validation-mlogloss:1.24183                                                 
[99]	validation-mlogloss:1.24166                                                 
[0]	validation-mlogloss:1.36304                                                  
[1]	validation-mlogloss:1.34574                                                  
[2]	validation-mlogloss:1.33141                                                  
[3]	validation-mlogloss:1.32023                                                  
[4]	validation-mlogloss:1.31164                                                  
[5]	validation-mlogloss:1.30458                                                  
[6]	validation-mlogloss:1.29943                                                  
[7]	validation-mlogloss:1.29449                                                  
[8]	validation-mlogloss:1.29036                                                  
[9]	validation-mlogloss:1.28743                                                  
[10]	validation-

[97]	validation-mlogloss:1.24876                                                 
[98]	validation-mlogloss:1.24887                                                 
[99]	validation-mlogloss:1.24899                                                 
[0]	validation-mlogloss:1.37750                                                  
[1]	validation-mlogloss:1.36989                                                  
[2]	validation-mlogloss:1.36259                                                  
[3]	validation-mlogloss:1.35586                                                  
[4]	validation-mlogloss:1.35017                                                  
[5]	validation-mlogloss:1.34471                                                  
[6]	validation-mlogloss:1.33976                                                  
[7]	validation-mlogloss:1.33482                                                  
[8]	validation-mlogloss:1.33043                                                  
[9]	validation-m

[96]	validation-mlogloss:1.26364                                                 
[97]	validation-mlogloss:1.26330                                                 
[98]	validation-mlogloss:1.26309                                                 
[99]	validation-mlogloss:1.26293                                                 
[0]	validation-mlogloss:1.35095                                                  
[1]	validation-mlogloss:1.32818                                                  
[2]	validation-mlogloss:1.31298                                                  
[3]	validation-mlogloss:1.30128                                                  
[4]	validation-mlogloss:1.29405                                                  
[5]	validation-mlogloss:1.28854                                                  
[6]	validation-mlogloss:1.28532                                                  
[7]	validation-mlogloss:1.28239                                                  
[8]	validation-m

...

[54]	validation-mlogloss:1.26140                                                 
[55]	validation-mlogloss:1.26114                                                 
[56]	validation-mlogloss:1.26074                                                 
[57]	validation-mlogloss:1.26061                                                 
[58]	validation-mlogloss:1.26038                                                 
[59]	validation-mlogloss:1.26000                                                 
[60]	validation-mlogloss:1.25989                                                 
[61]	validation-mlogloss:1.25951                                                 
[62]	validation-mlogloss:1.25931                                                 
[63]	validation-mlogloss:1.25899                                                 
[64]	validation-mlogloss:1.25882                                                 
[65]	validation-mlogloss:1.25853                                                 
[66]	validation-

[53]	validation-mlogloss:1.27536                                                 
[54]	validation-mlogloss:1.27505                                                 
[55]	validation-mlogloss:1.27467                                                 
[56]	validation-mlogloss:1.27433                                                 
[57]	validation-mlogloss:1.27378                                                 
[58]	validation-mlogloss:1.27359                                                 
[59]	validation-mlogloss:1.27333                                                 
[60]	validation-mlogloss:1.27301                                                 
[61]	validation-mlogloss:1.27263                                                 
[62]	validation-mlogloss:1.27228                                                 
[63]	validation-mlogloss:1.27206                                                 
[64]	validation-mlogloss:1.27187                                                 
[65]	validation-

[52]	validation-mlogloss:1.27404                                                 
[53]	validation-mlogloss:1.27352                                                 
[54]	validation-mlogloss:1.27312                                                 
[55]	validation-mlogloss:1.27272                                                 
[56]	validation-mlogloss:1.27236                                                 
[57]	validation-mlogloss:1.27199                                                 
[58]	validation-mlogloss:1.27165                                                 
[59]	validation-mlogloss:1.27127                                                 
[60]	validation-mlogloss:1.27090                                                 
[61]	validation-mlogloss:1.27053                                                 
[62]	validation-mlogloss:1.27016                                                 
[63]	validation-mlogloss:1.26986                                                 
[64]	validation-

[51]	validation-mlogloss:1.26841                                                 
[52]	validation-mlogloss:1.26803                                                 
[53]	validation-mlogloss:1.26778                                                 
[54]	validation-mlogloss:1.26735                                                 
[55]	validation-mlogloss:1.26696                                                 
[56]	validation-mlogloss:1.26681                                                 
[57]	validation-mlogloss:1.26653                                                 
[58]	validation-mlogloss:1.26628                                                 
[59]	validation-mlogloss:1.26597                                                 
[60]	validation-mlogloss:1.26552                                                 
[61]	validation-mlogloss:1.26526                                                 
[62]	validation-mlogloss:1.26503                                                 
[63]	validation-

[50]	validation-mlogloss:1.26776                                                 
[51]	validation-mlogloss:1.26713                                                 
[52]	validation-mlogloss:1.26693                                                 
[53]	validation-mlogloss:1.26665                                                 
[54]	validation-mlogloss:1.26636                                                 
[55]	validation-mlogloss:1.26619                                                 
[56]	validation-mlogloss:1.26607                                                 
[57]	validation-mlogloss:1.26592                                                 
[58]	validation-mlogloss:1.26581                                                 
[59]	validation-mlogloss:1.26564                                                 
[60]	validation-mlogloss:1.26546                                                 
[61]	validation-mlogloss:1.26517                                                 
[62]	validation-

In [40]:
import hyperopt
# Train the final model with the best hyperparameters
xgb_best = xgb.train(
    params=hyperopt.space_eval(xgb_space, xgb_best_result),
    dtrain=dtrain, 
    num_boost_round=100,
    evals=[(dval, 'validation')])

# Predict on the validation set with the best model
y_val_pred_xgb = xgb_best.predict(dval).astype(int)

# Generate the classification report
xgb_report = classification_report(y_val, y_val_pred_xgb)
print(xgb_report)


[0]	validation-mlogloss:1.37608
[1]	validation-mlogloss:1.36615
[2]	validation-mlogloss:1.35714
[3]	validation-mlogloss:1.34903
[4]	validation-mlogloss:1.34227
[5]	validation-mlogloss:1.33587
[6]	validation-mlogloss:1.33047
[7]	validation-mlogloss:1.32504
[8]	validation-mlogloss:1.31982
[9]	validation-mlogloss:1.31508
[10]	validation-mlogloss:1.31111
[11]	validation-mlogloss:1.30777
[12]	validation-mlogloss:1.30439
[13]	validation-mlogloss:1.30137
[14]	validation-mlogloss:1.29860
[15]	validation-mlogloss:1.29626
[16]	validation-mlogloss:1.29372
[17]	validation-mlogloss:1.29179
[18]	validation-mlogloss:1.28973
[19]	validation-mlogloss:1.28790
[20]	validation-mlogloss:1.28616
[21]	validation-mlogloss:1.28457
[22]	validation-mlogloss:1.28334
[23]	validation-mlogloss:1.28185
[24]	validation-mlogloss:1.28052
[25]	validation-mlogloss:1.27913
[26]	validation-mlogloss:1.27818
[27]	validation-mlogloss:1.27714
[28]	validation-mlogloss:1.27585
[29]	validation-mlogloss:1.27465
[30]	validation-mlog

In [41]:
# Model 2: gb
gb_best_result = fmin(
    fn=gradient_boosting_objective,
    space=gb_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)


 28%|██▊       | 14/50 [01:32<03:12,  5.35s/trial, best loss: 0.5883857145300394]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 30%|███       | 15/50 [01:35<02:39,  4.57s/trial, best loss: 0.5883857145300394]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



100%|██████████| 50/50 [07:43<00:00,  9.27s/trial, best loss: 0.5798865170756197]


In [42]:
# Retrieve the best hyperparameters from the Trials object
gb_best_params = hyperopt.space_eval(gb_space, gb_best_result)

# Train the final model with the best hyperparameters
gb_best = GradientBoostingClassifier(random_state=0, **gb_best_params)
gb_best.fit(X_train, y_train)

# Predict on the validation set with the best model
y_val_pred_gb = gb_best.predict(X_val).astype(int)

# Generate the classification report
gb_report = classification_report(y_val, y_val_pred_gb)
print(gb_report)



              precision    recall  f1-score   support

           0       0.65      0.19      0.29       559
           1       0.36      0.11      0.17       880
           2       0.37      0.12      0.18      1185
           3       0.41      0.90      0.57      1511

    accuracy                           0.41      4135
   macro avg       0.45      0.33      0.30      4135
weighted avg       0.42      0.41      0.33      4135



In [ ]:
# Model 3: lr
lr_best_result = fmin(
    fn=lr_objective,
    space=lr_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)


  6%|▍      | 3/50 [50:51<13:15:05, 1015.01s/trial, best loss: 0.589629335659444]

In [ ]:
# Retrieve the best hyperparameters from the Trials object
lr_best_params = hyperopt.space_eval(lr_space, lr_best_result)

# Train the final model with the best hyperparameters
lr_best = LogisticRegression(random_state=0, **lr_best_params)
lr_best.fit(X_train, y_train)

# Predict on the validation set with the best model
y_val_pred_lr = lr_best.predict(X_val).astype(int)

# Generate the classification report
lr_report = classification_report(y_val, y_val_pred_lr)
print(lr_report)



In [37]:
# Model 4: dt
dt_best_result = fmin(
    fn=dt_objective,
    space=dt_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)


  2%|▏           | 1/50 [00:00<00:36,  1.36trial/s, best loss: 0.765896675211543]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 34%|███▍      | 17/50 [00:14<00:25,  1.30trial/s, best loss: 0.6659208122046166]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 48%|████▊     | 24/50 [00:19<00:18,  1.41trial/s, best loss: 0.6535511346624179]

...

 88%|████████▊ | 44/50 [00:34<00:04,  1.42trial/s, best loss: 0.6224703396395346]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 98%|█████████▊| 49/50 [00:38<00:00,  1.34trial/s, best loss: 0.6224703396395346]

/home/tunglinux/.virtualenvs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



100%|██████████| 50/50 [00:38<00:00,  1.29trial/s, best loss: 0.6224703396395346]


In [38]:
dt_best_params = hyperopt.space_eval(dt_space, dt_best_result)

# Train the final model with the best hyperparameters
dt_best = DecisionTreeClassifier(random_state=0, **dt_best_params)
dt_best.fit(X_train, y_train)

# Predict on the validation set with the best model
y_val_pred_dt = dt_best.predict(X_val).astype(int)

# Generate the classification report
dt_report = classification_report(y_val, y_val_pred_dt)
print(dt_report)



              precision    recall  f1-score   support

           0       0.20      0.62      0.30       559
           1       0.26      0.04      0.07       880
           2       0.50      0.00      0.00      1185
           3       0.42      0.62      0.50      1511

    accuracy                           0.32      4135
   macro avg       0.34      0.32      0.22      4135
weighted avg       0.38      0.32      0.24      4135



Observations:
 - The model with high precision but not too low-overall accuracy: GB
- The model with highest recall rates (for higher classes): DT
- Most model do better on the extreme classes (0 and 3) than they do on middle classes (1 and 2)

# Conclusion
- We followed a very logical EDA which led to a GB model with 0.68 and 0.41 precision rate for higher classes (class 0 and class 1), of course at the cost of low recall.
- Recall is low but we must consider that we just used only the delivery data. I expect better performance if we have more finance related information about each importer
 


# What's next

I got to finish the assignment in 3 days, most of which spent cleaning datasets. There were more I wanted to do:

- feature selection: I used too many features, and did not visualize the importance of each.

- the shortened hs code contains 4 digits, but I can also try just 2 first (Chapter)

- graph clustering/community detection: We can think of the data as a graph where nodes are importers than hs codes, and the relationships can be the total calculated teu (may be scaled/normalized), etc.. This can reveal what kinds of importer they are: car producers, fruits retailers, etc... However, that might not be super helpful since such an information can be easily found somewhere else.

- hypergraph mining: Each line in `main.csv` can be think of a hyperedge of multiple entities: hs_code, importer, port, etc.. This requires advanced techniques that I'm currently not familiar with. 

- hs_code can also implies the tax rate, which can be useful as a feature